In [1]:
import tensorflow as tf
import numpy as np
import gym

np.random.seed(2)
tf.set_random_seed(2)  # reproducible


class Actor(object):
    def __init__(self, sess, n_features, action_bound, lr=0.0001):
        self.sess = sess

        self.s = tf.placeholder(tf.float32, [1, n_features], "state")
        self.a = tf.placeholder(tf.float32, None, name="act")
        self.td_error = tf.placeholder(tf.float32, None, name="td_error")  # TD_error

        l1 = tf.layers.dense(
            inputs=self.s,
            units=30,  # number of hidden units
            activation=tf.nn.relu,
            kernel_initializer=tf.random_normal_initializer(0., .1),  # weights
            bias_initializer=tf.constant_initializer(0.1),  # biases
            name='l1'
        )

        mu = tf.layers.dense(
            inputs=l1,
            units=1,  # number of hidden units
            activation=tf.nn.tanh,
            kernel_initializer=tf.random_normal_initializer(0., .1),  # weights
            bias_initializer=tf.constant_initializer(0.1),  # biases
            name='mu'
        )

        sigma = tf.layers.dense(
            inputs=l1,
            units=1,  # output units
            activation=tf.nn.softplus,  # get action probabilities
            kernel_initializer=tf.random_normal_initializer(0., .1),  # weights
            bias_initializer=tf.constant_initializer(1.),  # biases
            name='sigma'
        )
        global_step = tf.Variable(0, trainable=False)
        # self.e = epsilon = tf.train.exponential_decay(2., global_step, 1000, 0.9)
        self.mu, self.sigma = tf.squeeze(mu*2), tf.squeeze(sigma+0.1)
        self.normal_dist = tf.distributions.Normal(self.mu, self.sigma)

        self.action = tf.clip_by_value(self.normal_dist.sample(1), action_bound[0], action_bound[1])

        with tf.name_scope('exp_v'):
            log_prob = self.normal_dist.log_prob(self.a)  # loss without advantage
            self.exp_v = log_prob * self.td_error  # advantage (TD_error) guided loss
            # Add cross entropy cost to encourage exploration
            self.exp_v += 0.01*self.normal_dist.entropy()

        with tf.name_scope('train'):
            self.train_op = tf.train.AdamOptimizer(lr).minimize(-self.exp_v, global_step)    # min(v) = max(-v)

    def learn(self, s, a, td):
        s = s[np.newaxis, :]
        feed_dict = {self.s: s, self.a: a, self.td_error: td}
        _, exp_v = self.sess.run([self.train_op, self.exp_v], feed_dict)
        return exp_v

    def choose_action(self, s):
        s = s[np.newaxis, :]
        return self.sess.run(self.action, {self.s: s})  # get probabilities for all actions


class Critic(object):
    def __init__(self, sess, n_features, lr=0.01):
        self.sess = sess
        with tf.name_scope('inputs'):
            self.s = tf.placeholder(tf.float32, [1, n_features], "state")
            self.v_ = tf.placeholder(tf.float32, [1, 1], name="v_next")
            self.r = tf.placeholder(tf.float32, name='r')

        with tf.variable_scope('Critic'):
            l1 = tf.layers.dense(
                inputs=self.s,
                units=30,  # number of hidden units
                activation=tf.nn.relu,
                kernel_initializer=tf.random_normal_initializer(0., .1),  # weights
                bias_initializer=tf.constant_initializer(0.1),  # biases
                name='l1'
            )

            self.v = tf.layers.dense(
                inputs=l1,
                units=1,  # output units
                activation=None,
                kernel_initializer=tf.random_normal_initializer(0., .1),  # weights
                bias_initializer=tf.constant_initializer(0.1),  # biases
                name='V'
            )

        with tf.variable_scope('squared_TD_error'):
            self.td_error = tf.reduce_mean(self.r + GAMMA * self.v_ - self.v)
            self.loss = tf.square(self.td_error)    # TD_error = (r+gamma*V_next) - V_eval
        with tf.variable_scope('train'):
            self.train_op = tf.train.AdamOptimizer(lr).minimize(self.loss)

    def learn(self, s, r, s_):
        s, s_ = s[np.newaxis, :], s_[np.newaxis, :]

        v_ = self.sess.run(self.v, {self.s: s_})
        td_error, _ = self.sess.run([self.td_error, self.train_op],
                                          {self.s: s, self.v_: v_, self.r: r})
        return td_error

In [2]:
OUTPUT_GRAPH = False
MAX_EPISODE = 1000
MAX_EP_STEPS = 200
DISPLAY_REWARD_THRESHOLD = -100  # renders environment if total episode reward is greater then this threshold
RENDER = False  # rendering wastes time
GAMMA = 0.9
LR_A = 0.001    # learning rate for actor
LR_C = 0.01     # learning rate for critic

env = gym.make('Pendulum-v0')
env.seed(1)  # reproducible
env = env.unwrapped

N_S = env.observation_space.shape[0]
A_BOUND = env.action_space.high

sess = tf.Session()

actor = Actor(sess, n_features=N_S, lr=LR_A, action_bound=[-A_BOUND, A_BOUND])
critic = Critic(sess, n_features=N_S, lr=LR_C)

sess.run(tf.global_variables_initializer())

if OUTPUT_GRAPH:
    tf.summary.FileWriter("logs/", sess.graph)

for i_episode in range(MAX_EPISODE):
    s = env.reset()
    t = 0
    ep_rs = []
    while True:
        # if RENDER:
        env.render()
        a = actor.choose_action(s)
        
        s_, r, done, info = env.step(a)
        r /= 10

        td_error = critic.learn(s, r, s_)  # gradient = grad[r + gamma * V(s_) - V(s)]
        actor.learn(s, a, td_error)  # true_gradient = grad[logPi(s,a) * td_error]

        s = s_
        t += 1
        ep_rs.append(r)
        if t > MAX_EP_STEPS:
            ep_rs_sum = sum(ep_rs)
            if 'running_reward' not in globals():
                running_reward = ep_rs_sum
            else:
                running_reward = running_reward * 0.9 + ep_rs_sum * 0.1
            if running_reward > DISPLAY_REWARD_THRESHOLD: RENDER = True  # rendering
            print("episode:", i_episode, "  reward:", int(running_reward))
            break


/Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)
W0319 14:24:11.509199 4410860992 deprecation.py:323] From <ipython-input-1-413f876f4f31>:23: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.
W0319 14:24:11.548310 4410860992 deprecation.py:323] From <ipython-input-1-413f876f4f31>:46: Normal.__init__ (from tensorflow.python.ops.distributions.normal) is deprecated and will be removed after 2019-01-01.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
W0319 14:24:11.550029 4410860992 deprecation.py:323] From /Users

episode: 0   reward: -98
episode: 1   reward: -104
episode: 2   reward: -109
episode: 3   reward: -115
episode: 4   reward: -118
episode: 5   reward: -124
episode: 6   reward: -128
episode: 7   reward: -132
episode: 8   reward: -132
episode: 9   reward: -135
episode: 10   reward: -135
episode: 11   reward: -138
episode: 12   reward: -142
episode: 13   reward: -142
episode: 14   reward: -143
episode: 15   reward: -145
episode: 16   reward: -147
episode: 17   reward: -146
episode: 18   reward: -140
episode: 19   reward: -142
episode: 20   reward: -144
episode: 21   reward: -145
episode: 22   reward: -141
episode: 23   reward: -142
episode: 24   reward: -143
episode: 25   reward: -145
episode: 26   reward: -145
episode: 27   reward: -145
episode: 28   reward: -144
episode: 29   reward: -143
episode: 30   reward: -138
episode: 31   reward: -138
episode: 32   reward: -137
episode: 33   reward: -137
episode: 34   reward: -138
episode: 35   reward: -139
episode: 36   reward: -138
episode: 37 

episode: 300   reward: -107
episode: 301   reward: -106
episode: 302   reward: -113
episode: 303   reward: -117
episode: 304   reward: -120
episode: 305   reward: -114
episode: 306   reward: -111
episode: 307   reward: -112
episode: 308   reward: -117
episode: 309   reward: -114
episode: 310   reward: -118
episode: 311   reward: -115
episode: 312   reward: -119
episode: 313   reward: -123
episode: 314   reward: -121
episode: 315   reward: -125
episode: 316   reward: -121
episode: 317   reward: -117
episode: 318   reward: -119
episode: 319   reward: -116
episode: 320   reward: -113
episode: 321   reward: -109
episode: 322   reward: -109
episode: 323   reward: -114
episode: 324   reward: -103
episode: 325   reward: -100
episode: 326   reward: -99
episode: 327   reward: -89
episode: 328   reward: -91
episode: 329   reward: -90
episode: 330   reward: -92
episode: 331   reward: -95
episode: 332   reward: -92
episode: 333   reward: -89
episode: 334   reward: -96
episode: 335   reward: -103
e

episode: 596   reward: -111
episode: 597   reward: -115
episode: 598   reward: -112
episode: 599   reward: -109
episode: 600   reward: -109
episode: 601   reward: -105
episode: 602   reward: -112
episode: 603   reward: -118
episode: 604   reward: -117
episode: 605   reward: -120
episode: 606   reward: -123
episode: 607   reward: -120
episode: 608   reward: -118
episode: 609   reward: -108
episode: 610   reward: -99
episode: 611   reward: -95
episode: 612   reward: -101
episode: 613   reward: -103
episode: 614   reward: -109
episode: 615   reward: -100
episode: 616   reward: -102
episode: 617   reward: -108
episode: 618   reward: -103
episode: 619   reward: -109
episode: 620   reward: -108
episode: 621   reward: -113
episode: 622   reward: -118
episode: 623   reward: -117
episode: 624   reward: -121
episode: 625   reward: -119
episode: 626   reward: -122
episode: 627   reward: -119
episode: 628   reward: -122
episode: 629   reward: -121
episode: 630   reward: -121
episode: 631   reward:

episode: 889   reward: -129
episode: 890   reward: -125
episode: 891   reward: -121
episode: 892   reward: -120
episode: 893   reward: -120
episode: 894   reward: -123
episode: 895   reward: -127
episode: 896   reward: -127
episode: 897   reward: -124
episode: 898   reward: -126
episode: 899   reward: -125
episode: 900   reward: -124
episode: 901   reward: -124
episode: 902   reward: -127
episode: 903   reward: -129
episode: 904   reward: -129
episode: 905   reward: -128
episode: 906   reward: -127
episode: 907   reward: -130
episode: 908   reward: -127
episode: 909   reward: -124
episode: 910   reward: -127
episode: 911   reward: -123
episode: 912   reward: -122
episode: 913   reward: -121
episode: 914   reward: -121
episode: 915   reward: -121
episode: 916   reward: -126
episode: 917   reward: -123
episode: 918   reward: -126
episode: 919   reward: -124
episode: 920   reward: -127
episode: 921   reward: -125
episode: 922   reward: -125
episode: 923   reward: -129
episode: 924   rewar

In [3]:
total_steps = 0

for i_episode in range(10):

    observation = env.reset()
    ep_r = 0
    while True:
        env.render()

        action = RL.choose_action(observation)
        action = action * 2
        print(action)

        observation_, reward, done, info = env.step([action])
        print(observation_)

        theta_cos, theta_sin, theta_dot = observation_

        reward = - ((theta_dot)**2)

        RL.store_transition(observation, action, reward, observation_)

        if total_steps > 1000:
            RL.learn()

        ep_r += reward
        if done:
            print('episode: ', i_episode,
                  'ep_r: ', round(ep_r, 4),
                  'epsilon: ', round(RL.epsilon, 2))
            break

        observation = observation_
        total_steps += 1

RL.plot_cost()

NameError: name 'RL' is not defined